### 2.0 Notebook for training T5 model
**Hypothesis:** T5 model for translation can be used for text detoxification
All the running is made on colab, so paths are set up for colab. If you want to run it locally, you need to change the paths.

In [1]:
# ! pip install transformers[torch]
# ! pip install accelerate -U
# ! pip install accelerate>=0.20.1
# ! pip install datasets
# ! pip install transformers
# ! pip install sacrebleu
# ! pip install evaluate
# ! pip install wandb

!pip install -r /content/Text-de-toxification-PMLDL-IU/requirements.txt

!git clone https://github.com/ivancheroleg/Text-de-toxification-PMLDL-IU

Load the dataset from the local file

In [3]:
from datasets import load_from_disk

# load dataset from local file
dataset = load_from_disk("/content/Text-de-toxification-PMLDL-IU/data/interim/dataset")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 502214
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 27900
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 27900
    })
})

In [5]:
model_checkpoint = "t5-base"

from transformers import AutoTokenizer

# we will use autotokenizer for this purpose
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


### Preprocessing

In [6]:
prefix = "Translate toxic sentence to non-toxic sentence:"

max_input_length = 128
max_target_length = 128
source_sentence = "toxic"
target_sentence = "non-toxic"

def preprocess_function(examples):
    # Inputs
    inputs = [prefix + example[source_sentence] for example in examples["translation"]]
    targets = [example[target_sentence] for example in examples["translation"]]
    
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

preprocess_function(dataset["train"][:2])

{'input_ids': [[13959, 1566, 12, 3, 18609, 10, 99, 491, 4031, 8347, 7, 160, 28, 160, 2550, 2670, 6, 34, 133, 3209, 8, 306, 1425, 13, 6567, 7031, 1538, 449, 5, 1], [13959, 1566, 12, 3, 18609, 10, 4188, 31, 60, 2852, 27635, 53, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[3, 99, 491, 4031, 19, 18368, 160, 28, 26829, 2670, 6, 24, 3, 9453, 8, 306, 593, 13, 6567, 7031, 1538, 4849, 5, 1], [230, 25, 31, 60, 652, 23147, 5, 1]]}

### Metrics

In [8]:
from datasets import load_metric

sacrebleu = load_metric("sacrebleu")

<ipython-input-8-c70904755845>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  sacrebleu = load_metric("sacrebleu")


In [9]:
# for the example purpose I will crop the dataset and select first 25000 for train
# and 2500 for validation and test

cropped_datasets = dataset
cropped_datasets['train'] = dataset['train'].select(range(5000))
cropped_datasets['validation'] = dataset['validation'].select(range(500))
cropped_datasets['test'] = dataset['test'].select(range(500))
tokenized_datasets = cropped_datasets.map(preprocess_function, batched=True)

declare the model

In [10]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Create a model for the pretrained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Declare arguments for training
For visualization of training process we will use wandb (Weights and Biases) API. It will be used for logging the metrics and visualizing the training process.

In [11]:
# Defining the parameters for training
batch_size = 32
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_sentence}-to-{target_sentence}",
    evaluation_strategy = "epoch",
    learning_rate=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    report_to="wandb",
)

In [12]:
# instead of writing collate_fn function we will use DataCollatorForSeq2Seq
# similarly it implements the batch creation for training

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [13]:
import numpy as np

# simple postprocessing for text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

# compute metrics function to pass to trainer
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    sacrebleu_result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)

    result = {"bleu": sacrebleu_result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Garbage collection and emptying cuda cache

In [15]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

196

Seq2SeqTrainer

In [1]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

NameError: name 'Seq2SeqTrainer' is not defined

In [ ]:
trainer.train()

wandb: Currently logged in as: ivancholeg (ivanchteam). Use `wandb login --relogin` to force relogin


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


### Save the model and run inference

In [ ]:
# saving model
trainer.save_model('/content/Text-de-toxification-PMLDL-IU/models/t5_small_tuned')

In [ ]:
# loading the model and run inference for it
model = AutoModelForSeq2SeqLM.from_pretrained('/content/Text-de-toxification-PMLDL-IU/models/t5_small_tuned')
model.eval()
model.config.use_cache = False

### Evaluation
Here we can see the evaluation of the model on the test dataset

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

In [ ]:
trainer.save_model('/content/Text-de-toxification-PMLDL-IU/models/t5_small_tuned')

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('best')
model.eval()
model.config.use_cache = False

In [ ]:
def translate(model, inference_request, tokenizer=tokenizer):
    """
    Function for translation of the text
    :param model: given model
    :param inference_request: text to translate 
    :param tokenizer: tokenizer for the model
    :return: translated text
    """
    
    input_ids = tokenizer(inference_request, return_tensors="pt").input_ids
    outputs = model.generate(input_ids=input_ids)
    return tokenizer.decode(outputs[0], skip_special_tokens=True,temperature=0)

### Inference

In [ ]:
for i in range(10):
    text = cropped_datasets['train']['translation'][i]['toxic']
    print('-------------------')
    print(text)
    print(translate(model, text, tokenizer))